In [10]:
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
import ssl  #下载数据忽略证书

In [11]:
ssl._create_default_https_context = ssl._create_unverified_context

In [23]:
class FashionMnist:
    out_feature1 = 12  #第一组卷机核数量
    out_feature2 = 24  #第二组卷机核数量
    con_neuroons = 512  #第一个全联接神经元数量

    def __init__(self, path):
        self.data = read_data_sets(path, one_hot=True)
        self.sess = tf.Session()

    def init_weight_var(self, shape):
        '''根据指定形状初始化权重'''

        #截尾正态分布
        init_val = tf.truncated_normal(shape=shape,
                                       stddev=0.1)  #标准差
        return tf.Variable(init_val)

    def init_bias_var(self, shape):
        '''根据指定形状初始化偏置'''

        #截尾正态分布
        init_val = tf.constant(1.0, shape=shape)
        return tf.Variable(init_val)

    def conv2d(self, x, w):
        '''
        二维卷机
        :param x: 输入数据
        :param w: 卷机核
        :return: 卷机结果
        '''

        return tf.nn.conv2d(input=x,
                            filters=w,  #滤波器
                            strides=[1, 1, 1, 1],  #每个纬度上要有卷机，原始数据是4纬度的,翻译过来是每一张图，横向1，纵向1，1通道步幅度
                            padding='SAME')  #SAME可以做同纬卷机

    def max_pool_2x2(self, x):
        '''
        2x2区域最大池化
        :param x: 卷机结果
        :return: 池化结果
        '''

        return tf.nn.max_pool(value=x,  #输入数据
                              ksize=[1, 2, 2, 1],  #池化区域，每1张图 2x2区域 1通道
                              strides=[1, 2, 2, 1],  #池化步长
                              padding='SAME')  #步长超过原图，自适应补0

    def creat_conv_pool_layer(self, input, input_feature, out_feature):
        '''
        卷机池化组
        :param input: 输入数据
        :param input_feature: 输入特征数量
        :param out_feature: 输出特征数量
        :return: 返回计算结果
        '''
        filter_w = self.init_weight_var(
            shape=[5, 5, input_feature,
                   out_feature])  #一次只能处理一张图片，input_feature是1，因为是1通道的灰度图，翻译过来就是5x5的卷机，从input_feature个通道转化为out_feature通道，因为每次都只能处理一个图，所以表示第几张图的纬度不写
        b_conv = self.init_bias_var(shape=[out_feature])

        h_conv = tf.nn.relu(self.conv2d(x=input, w=filter_w) + b_conv)

        h_pool = self.max_pool_2x2(h_conv)

        return h_pool

    def creat_fc_layer(self, h_pool_flat, input_freature, con_neurons):
        '''
        第一个全链接
        :param h_pool_flat: 经过一纬度拉伸的数据
        :param input_freature: 输入特征数量
        :param con_neurons: 神经元数量
        :return: 返回计算结果
        '''

        w_fc = self.init_weight_var(shape=[input_freature, con_neurons])
        b_fc = self.init_bias_var([con_neurons])  #偏置数量等于神经元数量
        h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, w_fc) + b_fc)
        return h_fc1

    def build(self):
        '''
        组建cnn网络
        :return:
        '''
        #定义输入和输入占位符
        self.x = tf.placeholder(dtype=tf.float32, shape=[None, 784])
        self.y = tf.placeholder(dtype='float32', shape=[None, 10])
        x_image = tf.reshape(self.x, [-1, 28, 28, 1])  #调整图片尺寸

        #第一组卷机池化
        h_pool1 = self.creat_conv_pool_layer(input=x_image, input_feature=1, out_feature=self.out_feature1) # input_feature=1是因为输入的是灰度图，通道是1
        #第二组卷机池化
        h_pool2 = self.creat_conv_pool_layer(input=h_pool1, input_feature=self.out_feature1,
                                             out_feature=self.out_feature2)

        #全连接
        #输入特征数量
        h_pool2_flat_features = 7 * 7 * self.out_feature2
        #输入特征数据拉平，成了二维，第二个纬度表示有多数样本
        h_pool2_flat = tf.reshape(h_pool2, [-1, h_pool2_flat_features])  #-1是自适应样本数量
        h_fc1 = self.creat_fc_layer(h_pool_flat=h_pool2_flat,  #数据
                                    input_freature=h_pool2_flat_features,  #特征数量
                                    con_neurons=self.con_neuroons)  #神经元数量

        #丢弃层
        self.keep = tf.placeholder('float32')
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=self.keep)

        #输出层
        w_fc = self.init_weight_var([self.con_neuroons, 10])
        b_fc = self.init_bias_var([10])
        pred_y = tf.matmul(h_fc1_drop, w_fc) + b_fc

        #计算损失函数交叉熵，这个函数自动计算softmax
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=self.y,  #真实值
                                                       logits=pred_y)  #预测值
        cross_entropy = tf.reduce_mean(loss)

        #准确率
        corr_pred = tf.equal(tf.argmax(self.y, axis=1),  #真实类别
                             tf.argmax(pred_y, axis=1))  #预测类别
        self.acc = tf.reduce_mean(tf.cast(corr_pred, 'float32'))

        #优化器,自适应梯度下降优化器
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        self.train_op = optimizer.minimize(cross_entropy)

    def train(self):
        self.sess.run(tf.global_variables_initializer())

        batch_size = 100
        print('开始训练....')

        for i in range(10):
            total_batch = int(self.data.train.num_examples / batch_size)
            for j in range(total_batch):
                train_data = self.data.train.next_batch(batch_size=batch_size)
                params = {self.x: train_data[0], self.y: train_data[1], self.keep: 0.5}
                t, accuracy = self.sess.run([self.train_op, self.acc], feed_dict=params)

                if j % 100 == 0:
                    print('轮数:{},批次:{},精度:{}'.format(i, j, accuracy))

    def metrics(self, x, y, keep):

        params = {self.x: x, self.y: y, self.keep: keep}
        test_acc = self.sess.run(self.acc, feed_dict=params)
        print('测试集精度:', test_acc)

    def close(self):
        self.sess.close()


In [24]:
fmnist = FashionMnist('./fashionmnist/')  #下载数据
fmnist.build()
fmnist.train()
test_x, test_y = fmnist.data.test.next_batch(100)
fmnist.metrics(test_x, test_y, 0.5)
fmnist.close()

Extracting ./fashionmnist/train-images-idx3-ubyte.gz
Extracting ./fashionmnist/train-labels-idx1-ubyte.gz
Extracting ./fashionmnist/t10k-images-idx3-ubyte.gz
Extracting ./fashionmnist/t10k-labels-idx1-ubyte.gz
开始训练....
轮数:0,批次:0,精度:0.14000000059604645
轮数:0,批次:100,精度:0.7200000286102295
轮数:0,批次:200,精度:0.8899999856948853
轮数:0,批次:300,精度:0.8799999952316284
轮数:0,批次:400,精度:0.9300000071525574
轮数:0,批次:500,精度:0.9200000166893005
轮数:1,批次:0,精度:0.9800000190734863
轮数:1,批次:100,精度:0.9200000166893005
轮数:1,批次:200,精度:0.9300000071525574
轮数:1,批次:300,精度:0.9399999976158142
轮数:1,批次:400,精度:0.9599999785423279
轮数:1,批次:500,精度:0.9800000190734863
轮数:2,批次:0,精度:0.9700000286102295
轮数:2,批次:100,精度:0.9399999976158142
轮数:2,批次:200,精度:0.9599999785423279
轮数:2,批次:300,精度:0.9900000095367432
轮数:2,批次:400,精度:0.9800000190734863
轮数:2,批次:500,精度:0.9800000190734863
轮数:3,批次:0,精度:0.9700000286102295
轮数:3,批次:100,精度:0.9900000095367432
轮数:3,批次:200,精度:0.9800000190734863
轮数:3,批次:300,精度:1.0
轮数:3,批次:400,精度:0.9800000190734863
轮数:3,批次:500,精度:0.9800